In [1]:
import polars as pl
from transformers import AutoTokenizer

from pretrain_on_test._dum import _instruction_and_prompts
from pretrain_on_test import data

In [1]:
model_name = "hf-internal-testing/tiny-random-MistralForCausalLM"

In [2]:
import torch
from transformers import AutoModelForCausalLM, PreTrainedModel

In [3]:
model: PreTrainedModel = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

In [4]:
model.dtype

torch.float16

In [2]:
dataset_name = "ag_news"
path = f"/Users/kdubey/Downloads/run-2024-08-13_00-38-47-n500_contamination/accuracies/m100/n500/mistral-qlora-zero-shot/{dataset_name}/subsample_test_1.csv"
output_dataset_path = "dataset.json"  # will contain lines of prompt, completion

# Load stuff

In [3]:
classification_dataset_info = data.hf_dataset_name_to_classification_dataset_info[
    dataset_name
]

In [4]:
df = pl.read_csv(path)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")

# Test set (prompt, completion) pairs

In [6]:
class_names_unique = classification_dataset_info.class_names
instruction, prompts = _instruction_and_prompts(
    df["text"],
    tokenizer,
    class_names_unique,
    classification_dataset_info.task_description,
)

In [7]:
completions = [classification_dataset_info.class_names[label] for label in df["label"]]

In [8]:
prompts_completions = [
    f"{prompt} {completion}"
    for prompt, completion in zip(prompts, completions, strict=True)
]

In [9]:
print(prompts_completions[0])

### Text: Customers bemoan changes in Quicken 2005 The new version of the personal finance program drops support for a widely used file format.
 ### Answer: Sci/Tech


In [10]:
len(tokenizer.encode(prompts_completions[0]))

43

# Write so it can be plugged in for contamination analysis

In [11]:
import json

with open(output_dataset_path, "w") as f:
    json.dump(prompts_completions, f)

Check it makes sense

In [12]:
with open(output_dataset_path, "r") as f:
    data = f.read()
    examples = json.loads(data)

In [13]:
len(examples)

500

In [14]:
print(examples[0])

### Text: Customers bemoan changes in Quicken 2005 The new version of the personal finance program drops support for a widely used file format.
 ### Answer: Sci/Tech
